In [2]:
# !pip install dspy-ai
# import dspy

In [6]:
!pip install git+https://github.com/stanfordnlp/dspy.git
!pip install -U openai
import openai
!pip install datasets
!pip install joblib

from datasets import load_dataset

  Cloning https://github.com/stanfordnlp/dspy.git to /tmp/pip-req-build-pcueivpg
  Running command git clone --filter=blob:none --quiet https://github.com/stanfordnlp/dspy.git /tmp/pip-req-build-pcueivpg
  Resolved https://github.com/stanfordnlp/dspy.git to commit 4695bc763fc98d3ca16e677d28b9098480d17c0f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00
  C

In [7]:
from google.colab import userdata
api_key = userdata.get('api_key')

In [8]:
import dspy
model=dspy.OpenAI(model='gpt-3.5-turbo', api_key=api_key)

In [9]:
dspy.settings.configure(lm=model)

In [7]:
# medqa_dataset = load_dataset("bigbio/med_qa")
# print(medqa_dataset)

In [8]:
#PubMedQA
# pubmedqa_dataset = load_dataset("bigbio/pubmed_qa")
# print(pubmedqa_dataset)

In [9]:
#MedMCQA
# medmcqa_dataset = load_dataset("medmcqa")
# print(medmcqa_dataset)

In [10]:
#MMLU: clinical knowledge, medical genetics, anatomy, professional medicine, college biology, and college medicine. These are the datasets we will be using for our project.
# knowledges = ['clinical_knowledge','medical_genetics','anatomy', 'professional_medicine', 'college_biology', 'college_medicine']
# mmlu_dataset_professional = load_dataset("cais/mmlu",'professional_medicine')

In [10]:
medqa_dataset = load_dataset("bigbio/med_qa")

#training on the first 20 questions
train_subset = medqa_dataset["train"][:20]

train_questions = train_subset["question"]
train_answers_id = train_subset["answer_idx"]
train_options= train_subset["options"]
train_answer = train_subset["answer"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/27a6d96c9db3334f9b3c1f99484adac7b7579fe06926d41e9005000368623ad4/data_clean/questions/US/train.jsonl


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/27a6d96c9db3334f9b3c1f99484adac7b7579fe06926d41e9005000368623ad4/data_clean/questions/US/test.jsonl


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/27a6d96c9db3334f9b3c1f99484adac7b7579fe06926d41e9005000368623ad4/data_clean/questions/US/dev.jsonl


In [49]:
val_subset = medqa_dataset["validation"][:20]

val_questions = val_subset["question"]
val_answers_id = val_subset["answer_idx"]
val_options= val_subset["options"]
val_answer = val_subset["answer"]

formatted_val_options = [' '.join(f"{option['key']} {option['value']}" for option in options) for options in val_options]

In [6]:
print(medqa_dataset)

DatasetDict({
    train: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1272
    })
})


In [11]:
# # Data processing
formatted_train_options = [' '.join(f"{option['key']} {option['value']}" for option in options) for options in train_options]
# print(formatted_train_options)

In [12]:
# We are expecting a single choice answer so signature accordingly.
class MultipleChoiceQA(dspy.Signature):
    """Answer questions with single letter answers."""

    question = dspy.InputField(desc="The multiple-choice question.")
    options = dspy.InputField(desc="The set of options in the format : A option1 B option2 C option3 D option4 E option5 where A corresponds to option1, B to option2 and so on.")
    answer = dspy.OutputField(desc="A single-letter answer corresponding to the selected option.")

In [10]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [11]:
print(train_questions[1])
print(train_options[1])
print(train_answer[1])
print(train_answers_id)

A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?
[{'key': 'A', 'value': 'Placing the infant in a supine position on a firm mattress while sleeping'}, {'key': 'B', 'value': 'Routine postnatal electrocardiogram (ECG)'}, {'key': 'C', 'value': 'Keeping the infant covered and maintaining a high room temperature'}, {'key': 'D', 'value': 'Application of a device to maintain the sleeping position'}, {'key': 'E', 'value': 'Avoiding pacifier use during sleep'}]
Placing the infant in a supine position on a firm mattress while sleeping
['E', 'A', 'A', 'A', 'E', 'C', 'E', 'A', 'E', 'E', 'C', 'D', 'D', 'C', 'D', 'C', 'E', 'D', 'E', 'B']


In [18]:
# generate_answer = dspy.Predict(MultipleChoiceQA)
# dev_example=train_questions[0]
# pred = generate_answer(question=dev_example,options=formatted_train_options[0])
# print(f"Question: {dev_example}")
# print(f"Predicted Answer: {pred.answer}")

In [19]:
# generate_answer = dspy.Predict(BasicQA)
# dev_example='What is the capital of India?'
# pred = generate_answer(question=dev_example)
# print(f"Question: {dev_example}")
# print(f"Predicted Answer: {pred.answer}")

In [13]:
# Shuffling the questions.
import random
generate_answer = dspy.Predict(MultipleChoiceQA)

def generate_responses(questions: list[str], option_sets: list[str]) -> list[str]:
    responses = []
    combined_data = list(zip(questions, option_sets))
    random.shuffle(combined_data)
    shuffled_questions, shuffled_option_sets = zip(*combined_data)

    for question, options in zip(shuffled_questions, shuffled_option_sets):
        pred_response = generate_answer(question=question, options=options)
        generated_response = pred_response.answer
        responses.append(generated_response)

    return responses

# def generate_responses(questions, option_sets):
#     responses = []
#     for question, options_list in zip(questions, option_sets):
#         pred_response = generate_answer(question=question, options=options_list)
#         generated_response = pred_response.answer
#         responses.append(generated_response)
#     return responses

In [9]:
def test_generate_responses():
  assert generate_responses(['What are the symptoms of a headache?'],['A fever B cold C cough D high bp E migraine']) == ['E']
test_generate_responses()

In [14]:
def calculate_medqa_accuracy(ques, pred: list[str], true: list[str])-> float:
    correct_count = 0

    for p, t in zip(pred, true):
        if p == t:
            correct_count += 1

    accuracy = correct_count / len(pred)
    return accuracy

In [15]:
def validate_answer(ques, pred: list[str], ref: list[str])-> bool:
  return [p==t for p, t in zip(pred, ref)]

In [16]:
def create_train_set(train_questions, train_options, train_answers):
    train_set = []

    for question, options, answer in zip(train_questions, train_options, train_answers):
        example = dspy.Example(
            question=question,
            options=options,
            answer=answer
        ).with_inputs("question", "options")

        train_set.append(example)

    return train_set

In [30]:
def create_train_set_questions(train_questions):
    train_set = []

    for questions in train_questions:
        example = dspy.Example(
            question=questions,
        ).with_inputs("question")

        train_set.append(example)

    return train_set

In [31]:
train_set_questions = create_train_set_questions(train_questions)

In [51]:
val_set = create_train_set(val_questions, formatted_val_options, val_answers_id)

In [52]:
print(len(val_set))

20


In [46]:
print(train_set_questions)

[Example({'question': 'A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?'}) (input_keys={'question'}), Example({'question': 'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?'}) (input_keys={'question'}), Example({'question': "A mother brings her 3-week

In [18]:
train_set3 = create_train_set(train_questions, formatted_train_options, train_answers_id)

In [17]:
print(len(train_set3))

20


In [14]:
print(train_questions)
print(formatted_train_options)
print(train_answers_id)

['A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?', 'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?', "A mother brings her 3-week-old infant to the pediatrician's office because she is concerned about his feeding habits. He was born without complic

In [17]:
train_set = [dspy.Example(question="Which of the following is the primary function of insulin?", options="A Glycogen breakdown B Protein synthesis C Glucose uptake D Lipid digestion E Oxygen transport", answer="C").with_inputs("question","options"),
dspy.Example(question="What is the most common cause of acute pancreatitis?", options="A Alcohol consumption B Gallstones C Trauma D Viral infection E Genetic predisposition", answer="B").with_inputs("question","options"),
dspy.Example(question="Which of the following is a characteristic finding in rheumatoid arthritis?", options="A HLA-B27 positivity B Anti-dsDNA antibodies C Rheumatoid factor positivity D Anti-Smith antibodies E Decreased C-reactive protein", answer="C").with_inputs("question","options"),
dspy.Example(question="In a patient with chronic renal failure, which electrolyte imbalance is commonly observed?", options="A Hyperkalemia B Hyponatremia C Hypocalcemia D Hyperphosphatemia E Hypomagnesemia", answer="A").with_inputs("question","options"),
dspy.Example(question="Which class of antibiotics is known to cause irreversible ototoxicity?", options="A Penicillins B Cephalosporins C Aminoglycosides D Tetracyclines E Macrolides", answer="C").with_inputs("question","options"),
dspy.Example(question="What is the gold standard diagnostic test for acute appendicitis?", options="A Ultrasound B CT scan C MRI D X-ray E Physical examination", answer="B").with_inputs("question","options"),
dspy.Example(question="A patient with a history of frequent sun exposure presents with a nodular, pearly lesion on the face What is the most likely diagnosis?", options="A Psoriasis B Basal cell carcinoma C Melanoma D Squamous cell carcinoma E Rosacea", answer="B").with_inputs("question","options"),
dspy.Example(question="Which hormone is responsible for stimulating milk production in the mammary glands?", options="A Progesterone B Prolactin C Estrogen D Oxytocin E Human chorionic gonadotropin (hCG)", answer="B").with_inputs("question","options"),
dspy.Example(question="What is the first-line treatment for status epilepticus?", options="A Diazepam B Phenytoin C Levetiracetam D Lorazepam E Phenobarbital", answer="D").with_inputs("question","options"),
dspy.Example(question="A patient presents with chest pain that improves with leaning forward Which condition is most likely responsible for this symptom?", options="A Myocardial infarction B Pericarditis C Angina pectoris D Aortic dissection E Pulmonary embolism", answer="B").with_inputs("question","options")]

In [19]:
train_set2 = [
    dspy.Example(question="Which of the following is the primary function of insulin?", options="A. Glycogen breakdown B. Protein synthesis C. Glucose uptake D. Lipid digestion E. Oxygen transport", answer="C").with_inputs("question","options"),
    dspy.Example(question="What is the most common cause of acute pancreatitis?", options="A. Alcohol consumption B. Gallstones C. Trauma D. Viral infection E. Genetic predisposition", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a characteristic finding in rheumatoid arthritis?", options="A. HLA-B27 positivity B. Anti-dsDNA antibodies C. Rheumatoid factor positivity D. Anti-Smith antibodies E. Decreased C-reactive protein", answer="C").with_inputs("question","options"),
    dspy.Example(question="In a patient with chronic renal failure, which electrolyte imbalance is commonly observed?", options="A. Hyperkalemia B. Hyponatremia C. Hypocalcemia D. Hyperphosphatemia E. Hypomagnesemia", answer="A").with_inputs("question","options"),
    dspy.Example(question="Which class of antibiotics is known to cause irreversible ototoxicity?", options="A. Penicillins B. Cephalosporins C. Aminoglycosides D. Tetracyclines E. Macrolides", answer="C").with_inputs("question","options"),
    dspy.Example(question="What is the gold standard diagnostic test for acute appendicitis?", options="A. Ultrasound B. CT scan C. MRI D. X-ray E. Physical examination", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of frequent sun exposure presents with a nodular, pearly lesion on the face. What is the most likely diagnosis?", options="A. Psoriasis B. Basal cell carcinoma C. Melanoma D. Squamous cell carcinoma E. Rosacea", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which hormone is responsible for stimulating milk production in the mammary glands?", options="A. Progesterone B. Prolactin C. Estrogen D. Oxytocin E. Human chorionic gonadotropin (hCG)", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the first-line treatment for status epilepticus?", options="A. Diazepam B. Phenytoin C. Levetiracetam D. Lorazepam E. Phenobarbital", answer="D").with_inputs("question","options"),
    dspy.Example(question="A patient presents with chest pain that improves with leaning forward. Which condition is most likely responsible for this symptom?", options="A. Myocardial infarction B. Pericarditis C. Angina pectoris D. Aortic dissection E. Pulmonary embolism", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which vitamin deficiency is associated with bleeding gums and easy bruising?", options="A. Vitamin A B. Vitamin B12 C. Vitamin C D. Vitamin D E. Vitamin K", answer="C").with_inputs("question","options"),
    dspy.Example(question="In a patient with suspected pulmonary embolism, which imaging modality is preferred for initial evaluation?", options="A. Chest X-ray B. CT pulmonary angiography C. Ventilation-perfusion scan D. Magnetic resonance imaging (MRI) E. Ultrasound", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of alcohol abuse presents with confusion, ataxia, and ophthalmoplegia. What is the most likely diagnosis?", options="A. Wernicke's encephalopathy B. Korsakoff syndrome C. Delirium tremens D. Hepatic encephalopathy E. Cerebral contusion", answer="A").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a first-line treatment for hypertension in a patient with diabetes mellitus?", options="A. Atenolol B. Lisinopril C. Hydrochlorothiazide D. Diltiazem E. Clonidine", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the primary neurotransmitter involved in the pathophysiology of Alzheimer's disease?", options="A. Dopamine B. Acetylcholine C. Serotonin D. Gamma-aminobutyric acid (GABA) E. Norepinephrine", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of rheumatic heart disease presents with mitral valve stenosis. Which physical finding is characteristic of this condition?", options="A. Loud S1 heart sound B. Opening snap C. Holosystolic murmur D. Pulsus paradoxus E. Systolic ejection murmur", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the most common cause of viral myocarditis?", options="A. Coxsackievirus B. Influenza virus C. Human immunodeficiency virus (HIV) D. Herpes simplex virus E. Epstein-Barr virus", answer="A").with_inputs("question","options"),
    dspy.Example(question="Which of the following drugs is contraindicated in patients with a sulfa allergy?", options="A. Cephalexin B. Amoxicillin C. Ciprofloxacin D. Trimethoprim-sulfamethoxazole E. Doxycycline", answer="D").with_inputs("question","options"),
    dspy.Example(question="A patient presents with recurrent episodes of sudden-onset vertigo, hearing loss, and tinnitus. What is the most likely diagnosis?", options="A. Migraine-associated vertigo B. Meniere's disease C. Vestibular neuritis D. Benign paroxysmal positional vertigo (BPPV) E. Labyrinthitis", answer="B").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a characteristic feature of polycystic ovary syndrome (PCOS)?", options="A. Hyperprolactinemia B. Oligomenorrhea C. Hypogonadism D. Amenorrhea E. Increased estrogen levels", answer="B").with_inputs("question","options"),
    dspy.Example(question="A patient with chronic kidney disease is prescribed a phosphate binder. Which medication is commonly used for this purpose?", options="A. Calcium carbonate B. Sevelamer C. Furosemide D. Spironolactone E. Enalapril", answer="B").with_inputs("question","options"),
    dspy.Example(question="What is the classic triad of symptoms in diabetes insipidus?", options="A. Polyuria, polydipsia, weight loss B. Polyuria, polydipsia, hyperglycemia C. Polyuria, polydipsia, hypotension D. Polyuria, polydipsia, hypernatremia E. Polyuria, polydipsia, hypoglycemia", answer="D").with_inputs("question","options"),
    dspy.Example(question="A newborn presents with abdominal distension, vomiting, and failure to pass meconium. What is the most likely diagnosis?", options="A. Intussusception B. Hirschsprung disease C. Pyloric stenosis D. Meconium ileus E. Imperforate anus", answer="D").with_inputs("question","options"),
    dspy.Example(question="Which of the following is a common side effect of angiotensin-converting enzyme (ACE) inhibitors?", options="A. Hyperkalemia B. Hypokalemia C. Hypercalcemia D. Hyponatremia E. Hypoglycemia", answer="A").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of chronic alcohol abuse presents with bleeding tendencies. Which vitamin deficiency is most likely responsible for this clinical manifestation?", options="A. Vitamin A B. Vitamin B12 C. Vitamin C D. Vitamin D E. Vitamin K", answer="E").with_inputs("question","options"),
    dspy.Example(question="What is the first-line treatment for uncomplicated urinary tract infection (UTI) in a non-pregnant woman?", options="A. Ciprofloxacin B. Amoxicillin C. Trimethoprim-sulfamethoxazole D. Nitrofurantoin E. Doxycycline", answer="D").with_inputs("question","options"),
    dspy.Example(question="A patient presents with a severe headache, photophobia, and a positive Brudzinski sign. What is the most likely diagnosis?", options="A. Migraine headache B. Tension-type headache C. Cluster headache D. Subarachnoid hemorrhage E. Sinusitis", answer="D").with_inputs("question","options"),
    dspy.Example(question="Which type of hepatitis is most commonly transmitted through contaminated food and water?", options="A. Hepatitis A B. Hepatitis B C. Hepatitis C D. Hepatitis D E. Hepatitis E", answer="A").with_inputs("question","options"),
    dspy.Example(question="A patient with a history of chronic obstructive pulmonary disease (COPD) presents with increased dyspnea, wheezing, and decreased breath sounds. What is the most likely diagnosis?", options="A. Pneumonia B. Pulmonary embolism C. Pneumothorax D. Asthma exacerbation E. Pleural effusion", answer="C").with_inputs("question","options"),
    dspy.Example(question="Which medication is commonly used as a first-line treatment for acute gout attacks?", options="A. Naproxen B. Allopurinol C. Colchicine D. Methotrexate E. Prednisone", answer="C").with_inputs("question","options")
]

In [17]:
print(train_set)

[Example({'question': 'Which of the following is the primary function of insulin?', 'options': 'A Glycogen breakdown B Protein synthesis C Glucose uptake D Lipid digestion E Oxygen transport', 'answer': 'C'}) (input_keys={'question', 'options'}), Example({'question': 'What is the most common cause of acute pancreatitis?', 'options': 'A Alcohol consumption B Gallstones C Trauma D Viral infection E Genetic predisposition', 'answer': 'B'}) (input_keys={'question', 'options'}), Example({'question': 'Which of the following is a characteristic finding in rheumatoid arthritis?', 'options': 'A HLA-B27 positivity B Anti-dsDNA antibodies C Rheumatoid factor positivity D Anti-Smith antibodies E Decreased C-reactive protein', 'answer': 'C'}) (input_keys={'question', 'options'}), Example({'question': 'In a patient with chronic renal failure, which electrolyte imbalance is commonly observed?', 'options': 'A Hyperkalemia B Hyponatremia C Hypocalcemia D Hyperphosphatemia E Hypomagnesemia', 'answer': '

In [20]:
# Zero Shot
class ZERO(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.Predict(MultipleChoiceQA)

    def forward(self, question, options):
        answer = self.generate_answer(question=question,options=options)
# Using hard constrain. (Assert)
        dspy.Assert(len(answer) == 1,
        "Answer should comprise of exactly one character.")
# Using soft constraint. (Suggest)
        dspy.Suggest(len(answer) == 1,
        "Answer should comprise of exactly one character.")

        return answer


In [22]:
zero = ZERO()

In [21]:
# Chain of thought.
class COT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(MultipleChoiceQA)

    def forward(self, question, options):
        answer = self.generate_answer(question=question,options=options)

        return answer

In [38]:
cot = COT()

In [37]:
from dspy.evaluate import Evaluate
metric = dspy.evaluate.metrics.answer_exact_match

In [23]:
# Creating subset of randomly selected questions. (Random few shot)
import random
def random_few_shot_train_set(train_set, batch_size):
    return random.sample(train_set, batch_size)
batch_size = 2
random_batch = random_few_shot_train_set(train_set, batch_size)

In [60]:
# from dspy.teleprompt import KNNFewShot
# from dspy.predict.knn import KNN

# knn_teleprompter = KNNFewShot(KNN, 7, train_set)
# compiled_knn = knn_teleprompter.compile(BasicQABot(), trainset=trainset)

In [ ]:
# !pip install -U sentence-transformers

In [ ]:
# !apt-get install libomp-dev
# !pip install faiss-cpu

# import faiss

In [42]:
# #knnfew shot and chain of thought
# from dspy.teleprompt import Teleprompter
# from dspy.teleprompt import BootstrapFewShot
# from dspy.predict.knn import KNN
# from sentence_transformers import SentenceTransformer
# class KNNFewShot(Teleprompter):
#     def __init__(self, KNN, k, trainset):
#         self.KNN = KNN(k, trainset)

#     def compile(self, student, *, teacher=None, trainset, valset=None):
#         student_copy = student.reset_copy()

#         def forward_pass(*args, **kwargs):
#             knn_trainset = self.KNN(**kwargs)
#             few_shot_bootstrap = BootstrapFewShot()
#             compiled_program = few_shot_bootstrap.compile(cot, trainset = knn_trainset)
#             return compiled_program(**kwargs)

#         student_copy.forward = types.MethodType(forward_pass, student_copy)
#         return student_copy

In [44]:
from dspy.teleprompt import KNNFewShot
from dspy.predict.knn import KNN

knn_teleprompter = KNNFewShot(KNN, 7, train_set)
compiled_knn = knn_teleprompter.compile(cot, trainset=train_set)

In [56]:
from dspy.evaluate.evaluate import Evaluate
evaluator = Evaluate(devset=val_set, num_threads=1, display_progress=True, display_table=0)

In [57]:
evaluator(zero, metric=metric)


Average Metric: 10 / 20  (50.0): 100%|██████████| 20/20 [00:07<00:00,  2.74it/s]

Average Metric: 10 / 20  (50.0%)


50.0

In [58]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric = calculate_medqa_accuracy, max_rounds = 5)
compiled = teleprompter.compile(ZERO(), trainset=train_set)


100%|██████████| 10/10 [00:00<00:00, 534.66it/s]

100%|██████████| 10/10 [00:00<00:00, 704.77it/s]

100%|██████████| 10/10 [00:00<00:00, 745.44it/s]

100%|██████████| 10/10 [00:00<00:00, 765.98it/s]

100%|██████████| 10/10 [00:00<00:00, 1251.69it/s]

Bootstrapped 0 full traces after 10 examples in round 4.


In [59]:
evaluator(compiled, metric=metric)


Average Metric: 10 / 20  (50.0): 100%|██████████| 20/20 [00:08<00:00,  2.24it/s]

Average Metric: 10 / 20  (50.0%)


50.0

In [61]:
evaluator(compiled_knn, metric=metric)


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:01<00:07,  1.21s/it]

 29%|██▊       | 2/7 [00:02<00:06,  1.36s/it]

 43%|████▎     | 3/7 [00:03<00:04,  1.10s/it]

 57%|█████▋    | 4/7 [00:04<00:03,  1.17s/it]


Bootstrapped 4 full traces after 5 examples in round 0.



Average Metric: 1 / 1  (100.0):   5%|▌         | 1/20 [00:06<02:08,  6.74s/it]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:01<00:06,  1.04s/it]

 29%|██▊       | 2/7 [00:01<00:04,  1.22it/s]

 43%|████▎     | 3/7 [00:02<00:03,  1.07it/s]

 57%|█████▋    | 4/7 [00:03<00:02,  1.05it/s]


Bootstrapped 4 full traces after 5 examples in round 0.



Average Metric: 2 / 2  (100.0):  10%|█         | 2/20 [00:11<01:43,  5.75s/it]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:01<00:08,  1.47s/it]

 29%|██▊       | 2/7 [00:02<00:05,  1.06s/it]

 43%|████▎     | 3/7 [00:03<00:04,  1.15s/it]

 57%|█████▋    | 4/7 [00:04<00:03,  1.04s/it]


...


Average Metric: 12 / 19  (63.2):  95%|█████████▌| 19/20 [02:19<00:08,  8.39s/it]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:00<00:05,  1.09it/s]

 29%|██▊       | 2/7 [00:01<00:04,  1.13it/s]

 43%|████▎     | 3/7 [00:02<00:03,  1.20it/s]

 57%|█████▋    | 4/7 [00:05<00:03,  1.25s/it]


Bootstrapped 4 full traces after 5 examples in round 0.



Average Metric: 12 / 20  (60.0): 100%|██████████| 20/20 [02:26<00:00,  7.32s/it]

Average Metric: 12 / 20  (60.0%)


60.0

In [ ]:
model.inspect_history()

In [89]:
def compiled_zero(questions: list[str], option_sets: list[str]) -> list[str]:
    responses = []
    combined_data = list(zip(questions, option_sets))
    random.shuffle(combined_data)
    shuffled_questions, shuffled_option_sets = zip(*combined_data)

    for question, options in zip(shuffled_questions, shuffled_option_sets):
        pred_response = compiled(question=question, options=options)
        generated_response = pred_response.answer
        responses.append(generated_response)

    return responses

In [35]:
compiled("How are you?"," A okay B Fine")

Prediction(
    answer='B'
)

In [27]:
import random
def random_few_shot_train_set(train_set, batch_size):
    return random.sample(train_set, batch_size)
batch_size = 2
random_batch = random_few_shot_train_set(train_set, batch_size)
print(random_batch)

[Example({'question': 'What is the most common cause of acute pancreatitis?', 'options': 'A Alcohol consumption B Gallstones C Trauma D Viral infection E Genetic predisposition', 'answer': 'B'}) (input_keys={'question', 'options'}), Example({'question': 'A patient presents with chest pain that improves with leaning forward Which condition is most likely responsible for this symptom?', 'options': 'A Myocardial infarction B Pericarditis C Angina pectoris D Aortic dissection E Pulmonary embolism', 'answer': 'B'}) (input_keys={'question', 'options'})]
